# Prompt Engineering : Advanced Prompting avec OpenAI

Dans ce notebook, nous allons tester différentes techniques avancées de **prompt engineering**:
- **Zero-shot prompting**
- **Few-shot prompting**
- **Chain-of-thought** (CoT)
- **Self-refine** (ou auto-amélioration)

Nous utiliserons la **nouvelle API** de la bibliothèque `openai` (>=1.0.0) via la classe `OpenAI` et ses méthodes de chat (`client.chat.completions.create`).


In [1]:
# ============================
# Cellule 1 : Installation
# ============================

%pip install openai tiktoken python-dotenv
# Remarque : Aucune fin de ligne en commentaire pour éviter l'erreur


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ============================
# Cellule 2 : Configuration
# ============================

import os
import sys
from dotenv import load_dotenv

# Charger la configuration depuis le fichier .env (dans le répertoire parent GenAI/)
load_dotenv('../.env')

# On suppose que ton .env contient :
# OPENAI_API_KEY=sk-xxxxxx
# (ou autre variable si tu utilises Azure)
#
# Récupère la clé d'API
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Clé API introuvable. Vérifie ton fichier .env.")

# Mode batch pour exécution non-interactive (Papermill, tests automatisés)
# Détection automatique si exécution via Papermill ou si stdin non disponible
def is_interactive():
    """Détecte si l'exécution est interactive (terminal) ou batch (Papermill)"""
    try:
        # Check if running in Papermill
        import __main__
        if hasattr(__main__, '__file__') and 'papermill' in str(getattr(__main__, '__file__', '')).lower():
            return False
        # Check if stdin is available
        if not sys.stdin.isatty():
            return False
        return True
    except:
        return False

BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true" or not is_interactive()
print(f"Mode: {'BATCH' if BATCH_MODE else 'INTERACTIF'}")

In [ ]:
# ============================
# Cellule 3 : Client OpenAI
# ============================

import openai
from openai import OpenAI

# Charger le modèle depuis .env ou utiliser gpt-4o par défaut
# Note: gpt-4o-mini est conservé pour les exemples avec temperature/max_tokens
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")
MODEL_NAME = DEFAULT_MODEL

# Instanciation du client
client = OpenAI(
    api_key=api_key,
    # Tu peux configurer d'autres options si besoin
)

print("Client OpenAI initialisé avec succès !")
print(f"Modèle par défaut: {MODEL_NAME}")

### Rappel des différences entre Zero-shot, Few-shot, Chain-of-thought et Self-refine

1. **Zero-shot Prompting**  
   - Aucune instruction ou exemple préalable (à part la demande de l'utilisateur).  
   - Simple et direct, mais parfois moins précis ou cohérent.

2. **Few-shot Prompting**  
   - Fournir quelques exemples “input → output” pour guider la réponse.  
   - Permet de **spécifier le format**, le style, ou le contenu souhaité.  
   - Améliore significativement la qualité des réponses sur des tâches complexes.

3. **Chain-of-thought (CoT)**  
   - On **incite** le modèle à détailler son raisonnement étape par étape.  
   - Souvent utile pour des questions de logique, mathématiques, programmation ou raisonnement complexe.  
   - Peut **augmenter** la cohérence et la justesse de la réponse (mais attention à ne pas divulguer ces “étapes” si elles sont confidentielles).

4. **Self-refine**  
   - Demander au modèle de s’auto-critiquer puis de proposer une réponse améliorée.  
   - Mise en œuvre en plusieurs appels (réponse initiale, re-demande d’analyse, ré-énoncé final).  
   - Intéressant pour du code, des textes longs, ou des situations nécessitant un contrôle qualité.

---


In [4]:
# ============================
# Cellule 5 : Zero-shot
# ============================

prompt_1 = "Donne-moi 3 idées de recettes végétariennes à base de tomates."
response_1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_1}
    ],
    # Contrôle du style
    max_tokens=400,
    temperature=0.7  # plus la température est haute, plus c'est créatif
)

print("=== Zero-shot Prompt ===")
print(f"Prompt: {prompt_1}\n")
print("Réponse du modèle :\n")
print(response_1.choices[0].message.content)


=== Zero-shot Prompt ===
Prompt: Donne-moi 3 idées de recettes végétariennes à base de tomates.

Réponse du modèle :

Bien sûr ! Voici trois idées de recettes végétariennes à base de tomates :

### 1. **Tarte Tomates et Féta**
**Ingrédients :**
- Pâte brisée
- Tomates (cerises ou en tranches)
- Féta
- Olives noires
- Herbes de Provence
- Huile d'olive
- Sel et poivre

**Instructions :**
1. Préchauffez le four à 180°C.
2. Étalez la pâte brisée dans un moule à tarte.
3. Disposez les tomates sur la pâte, puis émiettez la féta par-dessus.
4. Ajoutez les olives, assaisonnez avec les herbes de Provence, le sel et le poivre.
5. Arrosez d'un filet d'huile d'olive et enfournez pendant 30-35 minutes, jusqu'à ce que les tomates soient bien rôties.

### 2. **Ratatouille**
**Ingrédients :**
- 2 tomates
- 1 courgette
- 1 aubergine
- 1 poivron (rouge ou jaune)
- 1 oignon
- 2 gousses d'ail
- Herbes de Provence
- Huile d'olive
- Sel et poivre

**Instructions :**
1. Coupez tous les légumes en dés.
2. Da

### Résultat du Zero-shot prompting

Le zero-shot prompting est la technique la plus simple : **aucun exemple préalable**, juste une instruction directe.

**Avantages** :
- Rapide à mettre en œuvre
- Fonctionne bien pour des tâches courantes (résumés, traductions, questions générales)
- Économique en tokens

**Limites** :
- Moins précis sur des tâches complexes ou spécialisées
- Le format de sortie peut être imprévisible
- Nécessite des prompts très clairs et bien formulés

Dans cet exemple, le modèle génère 3 recettes végétariennes à base de tomates sans aucun exemple préalable. La qualité dépend fortement de la clarté du prompt et de la capacité du modèle à comprendre le domaine.

Ici, pas d’exemples ni d’instructions détaillées, on se contente d’un prompt direct.



###  Exemple Few-shot prompting (Code)

### Analyse du Few-shot prompting

Le few-shot prompting apporte une **amélioration significative** par rapport au zero-shot :

**Mécanisme** :
1. On fournit 2-3 exemples de la tâche souhaitée (paires question/réponse)
2. Le modèle apprend le **pattern** et le **format** attendu
3. Il applique ensuite ce pattern à la nouvelle question

**Avantages observables** :
- **Format cohérent** : Le modèle reproduit la structure des exemples (sujet, salutation, corps, signature)
- **Ton approprié** : Le style professionnel est maintenu
- **Contenu pertinent** : La réponse suit les conventions des exemples fournis

**Quand utiliser Few-shot ?**
- Tâches avec un format spécifique (emails, rapports, analyses structurées)
- Cas où le zero-shot donne des résultats trop variables
- Besoin de cohérence stylistique

**Compromis** : Chaque exemple consomme des tokens supplémentaires, donc à utiliser avec modération pour des contextes très longs.

In [5]:
# ============================
# Cellule X (NOUVELLE) : Few-shot supplémentaire
# ============================

few_shot_prompt_2 = """
Tu es un assistant spécialisé en rédaction d'e-mails professionnels.
Voici quelques exemples de style :

Exemple 1:
Q: Rédige un e-mail pour informer un client d'un retard de livraison
A: 
Sujet: Information concernant le retard de votre livraison

Bonjour [Nom du Client],

Nous tenions à vous informer que votre commande #1234 a pris du retard...
[...suite du mail...]

Exemple 2:
Q: Envoie un e-mail de remerciement pour un entretien d'embauche
A:
Sujet: Remerciements suite à notre entretien

Bonjour [Nom du Contact],

Je tiens à vous remercier pour le temps que vous m'avez accordé...
[...suite du mail...]

Maintenant, voici ma demande:

Q: Écris un e-mail pour informer un collaborateur d'un changement de planning et l'inviter à une réunion de suivi.
A:
"""

response_few_shot_2 = client.chat.completions.create(
    model=MODEL_NAME,  # ex. "gpt-4o-mini"
    messages=[
        {"role": "user", "content": few_shot_prompt_2}
    ],
    max_tokens=300,
    temperature=0.6
)

print("=== Exemple Few-shot (e-mail professionnel) ===")
print(response_few_shot_2.choices[0].message.content)


=== Exemple Few-shot (e-mail professionnel) ===
Sujet: Changement de planning et invitation à une réunion de suivi

Bonjour [Nom du Collaborateur],

Je souhaite vous informer d'un changement concernant notre planning initial. En raison de [mentionner la raison, si possible], nous devons ajuster certaines dates.

Pour discuter de ces modifications et de leur impact sur nos projets, je vous invite à une réunion de suivi qui se tiendra le [date] à [heure]. Nous pourrons ainsi aborder les nouvelles échéances et répondre à toutes vos questions.

Merci de me confirmer votre disponibilité pour cette réunion.

Cordialement,

[Votre Nom]  
[Votre Poste]  
[Votre Société]  
[Vos Coordonnées]


### Interprétation du Chain-of-thought

Le Chain-of-thought (CoT) est particulièrement efficace pour les **problèmes de raisonnement** :

**Analyse du résultat** :
Le modèle devrait avoir détaillé :
1. État initial : Alice a 5 pommes
2. Étape 1 : Elle en jette 2 → 5 - 2 = 3 pommes
3. Étape 2 : Elle en donne 1 à Bob → 3 - 1 = 2 pommes
4. Étape 3 : Bob lui rend 1 pomme → 2 + 1 = 3 pommes
5. **Réponse finale : 3 pommes**

**Pourquoi c'est important ?**
- **Transparence** : On peut vérifier le raisonnement étape par étape
- **Détection d'erreurs** : Si le résultat est faux, on peut identifier où le modèle s'est trompé
- **Confiance** : Le raisonnement explicite augmente la crédibilité
- **Debugging** : Facilite la correction du prompt si nécessaire

**Applications** :
- Calculs mathématiques
- Raisonnement logique
- Résolution de problèmes complexes
- Analyse de cas juridiques ou médicaux

**Note** : Temperature=0.2 garantit un raisonnement cohérent et reproductible.

Ici, nous donnons au modèle **deux exemples** de questions/réponses avant la **véritable question**. Cela oriente le style et le contexte.



###  7 : Exemple Chain-of-thought (Code)

On va demander un **calcul** simple, en guidant le modèle à réfléchir pas à pas :



- **Masquer le raisonnement si nécessaire** :  
  Parfois, on ne souhaite pas afficher au client final les étapes du raisonnement. Il existe des techniques pour “cacher” ce raisonnement ou n’afficher qu’un résumé. Par exemple :  
  1. Dans le prompt, on peut demander : “Don’t reveal your chain-of-thought. Provide only the final concise answer to the user.”  
  2. Ou effectuer un 2e appel API où l’on transmet l’enchaînement de pensées, mais on n'affiche que la conclusion.

- **Adapter la température** :  
  Pour un problème logique ou mathématique, une température trop élevée peut introduire des dérives ou des incohérences. Une température entre 0.0 et 0.3 est souvent recommandée pour les questions de calcul.

- **Chain-of-thought partiel** :  
  On peut encourager un raisonnement **intermédiaire** (quelques étapes clés) au lieu d’un raisonnement hyper détaillé, afin d’éviter que le texte devienne trop long ou difficile à comprendre.

- **Expliquer la démarche** :  
  On peut terminer par un résumé du raisonnement en 2-3 phrases, pour rendre la solution plus lisible.


In [6]:
# ============================
# Cellule 7 : Chain-of-thought
# ============================

cot_prompt = """
Alice a 5 pommes, elle en jette 2, puis elle en donne 1 à Bob.
Bob lui rend ensuite 1 pomme.
Combien de pommes Alice a-t-elle à la fin ?
Donne directement la réponse sans étape intermédiaire.
"""

# Explique ton raisonnement étape par étape, puis donne la réponse finale.
response_3 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": cot_prompt}
    ],
    max_tokens=200,
    temperature=0.2  # on réduit la température pour moins de fantaisie
)

print("=== Chain-of-thought Prompt ===")
print("Réponse du modèle (avec raisonnement) :\n")
print(response_3.choices[0].message.content)


=== Chain-of-thought Prompt ===
Réponse du modèle (avec raisonnement) :

Alice a 3 pommes à la fin.


On demande explicitement « explique ton raisonnement ». Cela **n’oblige** pas le modèle à le faire, mais en pratique, GPT-4o-mini (ou tout modèle qui gère le CoT) fournit souvent une solution pas-à-pas.

---

###  8 : Exemple Self-refine (Code)

L’idée : on fait **une première demande** (première réponse) et ensuite **on redemande** au modèle de s’auto-corriger.

#### 8a. Premier prompt

### Bilan du Self-refine

Le processus Self-refine en deux étapes démontre la capacité du modèle à s'**auto-améliorer** :

**Première étape** : Génération intentionnellement bugguée
- Le modèle crée du code avec un bug volontaire (par exemple, division par zéro, mauvaise initialisation, etc.)

**Deuxième étape** : Critique et correction
- Le modèle analyse son propre code
- Identifie le bug
- Propose une version corrigée
- Explique la nature du problème

**Enseignements** :
1. **Méta-cognition** : Le LLM peut raisonner sur ses propres productions
2. **Amélioration itérative** : Chaque passe peut affiner la qualité
3. **Détection de bugs** : Utile pour du code review automatisé

**Applications pratiques** :
- Génération de code robuste (plusieurs passes de correction)
- Rédaction de documents (brouillon → révision → version finale)
- Traduction (première traduction → révision → amélioration)

**Limite** : Chaque itération consomme des tokens et du temps. À utiliser pour des tâches critiques nécessitant haute qualité.

In [7]:
# ============================
# Cellule 8a : Self-refine Step 1
# ============================

prompt_sr1 = """
Ecris une courte fonction Python pour calculer la somme d'une liste. 
Ajoute un bug volontaire dans le code. 
"""

response_sr1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr1}],
    max_tokens=300
)

buggy_code = response_sr1.choices[0].message.content

print("=== Self-refine (1) : Code buggy ===\n")
print(buggy_code)


=== Self-refine (1) : Code buggy ===

Voici une fonction Python qui calcule la somme d'une liste mais qui contient un bug intentionnel :

```python
def somme_liste(ma_liste):
    total = 0
    for i in range(len(ma_liste)):
        total += ma_liste[i]  # Bug : on ajoute 1 au total à chaque itération
    return total + 1  # Bug : correction erronée
```

Ce code a un bug car il ajoute 1 au `total` à chaque itération de la boucle, ce qui fausse le résultat de la somme. De plus, il ajoute également 1 à la fin, ce qui rend le résultat encore plus incorrect.


#### 8b. Self-critique et amélioration

In [8]:
# ============================
# Cellule 8b : Self-refine Step 2
# ============================

prompt_sr2 = f"""
Voici un code Python qui contient un bug:

{buggy_code}

Peux-tu l'analyser, détecter le bug, proposer un correctif et une version améliorée du code ? 
Explique la correction.
"""

response_sr2 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr2}],
    max_tokens=400,
    temperature=0.3
)

print("=== Self-refine (2) : Correction ===\n")
print(response_sr2.choices[0].message.content)


=== Self-refine (2) : Correction ===

Analysons le code fourni :

```python
def somme_liste(ma_liste):
    total = 0
    for i in range(len(ma_liste)):
        total += ma_liste[i]  # Bug : on ajoute 1 au total à chaque itération
    return total + 1  # Bug : correction erronée
```

### Analyse du code

1. **Erreur dans l'addition** : Le commentaire indique qu'il y a un bug où 1 est ajouté à chaque itération. Cependant, le code ne fait pas cela. Il additionne simplement les éléments de `ma_liste` à `total`. Donc, le commentaire est trompeur. Le vrai problème est que la ligne `return total + 1` ajoute 1 au total final, ce qui est incorrect.

2. **Correction nécessaire** : Pour corriger le code, il suffit de retirer l'ajout de 1 lors du retour de la fonction. La somme des éléments de la liste doit être renvoyée telle quelle.

### Correctif proposé

Voici la version corrigée du code :

```python
def somme_liste(ma_liste):
    total = 0
    for i in range(len(ma_liste)):
        total += m

In [9]:
# ============================
# Cellule X (NOUVELLE) : Self-refine avec developer role
# ============================

messages_sr = [
    {
        "role": "developer",
        "content": (
            "You are a self-improving coding assistant. Whenever you provide code, "
            "you will automatically search for potential bugs or improvements "
            "and refine your output."
        )
    },
    {
        "role": "user",
        "content": (
            "Écris une fonction Python qui calcule la factorielle d'un nombre entier. "
            "Ensuite, relis-toi et corrige d'éventuels bugs."
        )
    }
]

response_self_refine = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages_sr,
    max_tokens=300,
    temperature=0.3
)

print("=== Self-refine avec 'developer role' ===")
print(response_self_refine.choices[0].message.content)


=== Self-refine avec 'developer role' ===
Voici une fonction Python qui calcule la factorielle d'un nombre entier. Je vais également vérifier le code pour d'éventuels bugs ou améliorations.

```python
def factorielle(n):
    if not isinstance(n, int):
        raise ValueError("L'argument doit être un entier.")
    if n < 0:
        raise ValueError("La factorielle n'est pas définie pour les nombres négatifs.")
    
    resultat = 1
    for i in range(2, n + 1):
        resultat *= i
    return resultat
```

### Vérifications et améliorations :

1. **Type de l'argument** : La fonction vérifie si l'argument est un entier, ce qui est une bonne pratique.
2. **Gestion des nombres négatifs** : La fonction lève une exception si `n` est négatif, ce qui est correct.
3. **Utilisation de `range`** : L'utilisation de `range(2, n + 1)` est efficace pour calculer la factorielle.
4. **Performance** : Pour des valeurs de `n` très élevées, il serait plus efficace d'utiliser une approche récursive ou d'

Ici, on utilise la première réponse pour nourrir le second prompt, demandant au modèle de **critiquer** et **améliorer** la réponse initiale.

---

### Cellule 9 : Interactive Prompt (Code)

Enfin, on peut proposer une **cellule interactive** : l’utilisateur peut saisir un prompt, et on envoie la requête au modèle :

In [ ]:
# ============================
# Cellule 9 : Prompt interactif
# ============================

# En mode batch (BATCH_MODE=true dans .env), cette cellule utilise des exemples prédéfinis
# En mode interactif, elle permet de tester le modèle en boucle

if BATCH_MODE:
    # Mode batch: exécuter des exemples simples
    test_prompts = ["Bonjour!", "Quelle est la capitale de la France?"]
    for prompt in test_prompts:
        print(f"[BATCH] Prompt: {prompt}")
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role":"user","content":prompt}],
            max_tokens=200,
            temperature=0.7
        )
        print(f"Réponse: {resp.choices[0].message.content}\n")
    print("Mode batch terminé.")
else:
    while True:
        user_input = input("Tape ton prompt ('exit' pour quitter) : ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("Fin de l'interaction.")
            break

        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role":"user","content":user_input}],
            max_tokens=200,
            temperature=0.7
        )

        print("\n=== Réponse du modèle ===")
        print(resp.choices[0].message.content)
        print("---------------------------------------------------\n")

In [ ]:
# ============================
# Cellule 9 : Prompt interactif avec mémoire de chat
# ============================

# En mode batch, cette cellule utilise un dialogue prédéfini
# En mode interactif, elle permet un échange avec mémoire

if BATCH_MODE:
    # Mode batch: simuler une conversation avec mémoire
    batch_conversation = [
        "Je m'appelle Alice",
        "Comment je m'appelle?",
    ]
    current_messages = []

    for user_input in batch_conversation:
        print(f"[BATCH] User: {user_input}")
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=current_messages + [{"role":"user","content":user_input}],
            max_tokens=200,
            temperature=0.7,
        )
        assistant_message = resp.choices[0].message.content
        print(f"Assistant: {assistant_message}\n")
        current_messages.append({"role":"user","content":user_input})
        current_messages.append({"role":"assistant","content":assistant_message})

    print("Mode batch (mémoire) terminé.")
else:
    user_input = ""
    current_messages = []

    while True:
        user_input = input("Tape ton prompt ('exit' pour quitter) : ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("Fin de l'interaction.")
            break
        print("\n=== message de l'utilisateur ===")
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages = current_messages + [{"role":"user","content":user_input}],
            max_tokens=200,
            temperature=0.7,
        )

        print("\n=== Réponse du modèle ===")
        assistant_message = resp.choices[0].message.content
        print(assistant_message)
        print("---------------------------------------------------\n")
        current_messages.append({"role":"user","content":user_input})
        current_messages.append({"role":"assistant","content":assistant_message})

# Prompting pour Modèles de Raisonnement (2025)

Les modèles de raisonnement (o4-mini, gpt-5-thinking) représentent une évolution majeure. Contrairement aux modèles de chat, ils prennent le temps de "réfléchir" avant de répondre.

## Différences clés avec les modèles chat

| Aspect | Modèles Chat (gpt-4o) | Modèles Raisonnement (o4-mini) |
|--------|----------------------|-------------------------------|
| Temps de réponse | Rapide | Plus lent (réflexion) |
| Prompts | Détaillés, structurés | **Simples et directs** |
| Chain-of-thought | Demandé explicitement | **Intégré nativement** |
| Messages | system, user, assistant | **developer**, user, assistant |
| Paramètre spécial | temperature | **reasoning_effort** |

## Règle d'or : Simplifier les prompts

**Pour les modèles de raisonnement, des prompts simples fonctionnent mieux !**

Les modèles raisonnants sont capables de :
- Comprendre l'intention sans instructions détaillées
- Gérer les ambiguïtés intelligemment
- Demander des clarifications si nécessaire

**Éviter** : "Analyse ce problème en détaillant chaque étape de ton raisonnement..."
**Préférer** : "Résous ce problème."

In [15]:
# ============================
# Cellule : Comparaison Chat vs Reasoning
# ============================

import time

probleme_complexe = """
Un fermier veut traverser une rivière avec un loup, une chèvre et un chou.
Son bateau ne peut transporter que lui et un objet à la fois.
Si le loup est laissé seul avec la chèvre, il la mange.
Si la chèvre est laissée seule avec le chou, elle le mange.
Comment le fermier peut-il tout transporter de l'autre côté?
"""

# Test avec gpt-4o-mini (chat model)
print("=== gpt-4o-mini (Chat Model) ===")
start = time.time()
response_chat = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": probleme_complexe}],
    max_tokens=500,
    temperature=0.2
)
print(f"Temps: {time.time() - start:.2f}s")
print(response_chat.choices[0].message.content[:400] + "...")

# Test avec o4-mini (reasoning model) - si disponible
print("\n=== o4-mini (Reasoning Model) ===")
try:
    start = time.time()
    response_reason = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "developer", "content": "Formatting re-enabled"},
            {"role": "user", "content": probleme_complexe}
        ],
        reasoning_effort="medium"
    )
    print(f"Temps: {time.time() - start:.2f}s")
    print(response_reason.choices[0].message.content[:400] + "...")
except Exception as e:
    print(f"o4-mini non disponible: {type(e).__name__}")
    print("Les modèles de raisonnement nécessitent un accès spécifique.")

=== gpt-4o-mini (Chat Model) ===
Temps: 5.36s
Le fermier peut transporter le loup, la chèvre et le chou de l'autre côté de la rivière en suivant ces étapes :

1. **Prendre la chèvre** et la transporter de l'autre côté de la rivière. Il laisse le loup et le chou ensemble, car ils ne se mangent pas.
   
2. **Retourner seul** de l'autre côté de la rivière.

3. **Prendre le loup** et le transporter de l'autre côté de la rivière.

4. **Laisser le ...

=== o4-mini (Reasoning Model) ===
Temps: 7.56s
Voici une façon de procéder en 7 traversées, sans jamais laisser ensemble le loup et la chèvre, ni la chèvre et le chou :

1. Le fermier prend la chèvre et la traverse sur l’autre rive.  
2. Il revient seul sur la rive de départ.  
3. Il prend le loup et le traverse sur l’autre rive.  
4. Il laisse le loup, reprend la chèvre et revient sur la rive de départ.  
5. Il prend le chou et le traverse su...


Maintenant, tu peux saisir n’importe quel prompt, et tu verras la réponse du modèle.  
Tape `exit` pour quitter la boucle.

----

## Conclusion et Ressources Supplémentaires

Dans ce notebook, nous avons approfondi diverses techniques de **prompt engineering** : 
- Zero-shot  
- Few-shot  
- Chain-of-thought  
- Self-refine  
- Interactions multi-messages (avec les rôles `system`, `developer`, `user`, `assistant`)  

### Ressources conseillées
- [**OpenAI Cookbook**](https://github.com/openai/openai-cookbook) : Recettes et astuces pour résoudre des problèmes concrets (prompt engineering, RAG, etc.).  
- [**Prompt Engineering Guide**](https://www.promptingguide.ai/) : Conseils de rédaction de prompts, cas d’usages, bonnes pratiques.  
- [**Chaine d’outils** (LangChain, LlamaIndex, etc.)](https://github.com/hwchase17/langchain) : Facilite la création de pipelines complexes (RAG, function calling, mémoire de conversation).  

> **Idées d’exercices**  
> 1. Adapter la technique Few-shot à d’autres cas (e.g., Q&R sur la finance, la santé ou le marketing).  
> 2. Utiliser la Self-refine pour générer un texte marketing, puis le réécrire en style “plus formel” ou “plus humoristique”.  
> 3. Tester la **combinaison** de techniques : un prompt Few-shot + Chain-of-thought + un re-run Self-refine.

---

Merci d'avoir suivi ce notebook sur le **prompt engineering avancé** !
